<a href="https://colab.research.google.com/github/Meilism/SpeechLM_finetuning/blob/main/SpeechLM/notebooks/speechlm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up environment

## Clone git repo and install fairseq editable

In [ ]:
root_dir = '/content'

%cd {root_dir}
!git clone https://github.com/Meilism/SpeechLM_finetuning.git

%cd SpeechLM_finetuning/SpeechLM
!git clone https://github.com/facebookresearch/fairseq.git

%cd fairseq
!git checkout 0338cdc3094ca7d29ff4d36d64791f7b4e4b5e6e
!pip install --editable .

In [ ]:
!pip install tensorboardX soundfile gdown editdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.9 MB/s eta 0:00:00


## Download pre-trained checkpoint and dataset to google drive

In [ ]:
# %cd {root_dir}
# !gdown https://drive.google.com/u/1/uc?id=1iJvhSGghNrMT-wAY1nwVu2YaYuTy1pxx&export=download
# !wget https://us.openslr.org/resources/12/train-clean-100.tar.gz
# !wget https://us.openslr.org/resources/12/dev-other.tar.gz
# !cp speechlmp_base_checkpoint_298_400000.pt /content/drive/MyDrive/TTIC-Speech/pre-trained/
# !cp train-clean-100.tar.gz /content/drive/MyDrive/TTIC-Speech/dataset/
# !cp dev-other.tar.gz /content/drive/MyDrive/TTIC-Speech/dataset/

/content
Downloading...
From: https://drive.google.com/u/1/uc?id=1iJvhSGghNrMT-wAY1nwVu2YaYuTy1pxx
To: /content/speechlmp_base_checkpoint_298_400000.pt
100% 397M/397M [00:03<00:00, 110MB/s]
--2023-05-30 00:49:01--  https://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  26.0MB/s    in 4m 4s   

2023-05-30 00:53:05 (24.9 MB/s) - ‘train-clean-100.tar.gz’ saved [6387309499/6387309499]

--2023-05-30 00:53:05--  https://us.openslr.org/resources/12/dev-other.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314305928 (300M

In [ ]:
%cd {root_dir}
!tar -xvzf /content/drive/MyDrive/TTIC-Speech/dataset/train-clean-100.tar.gz
!tar -xvzf /content/drive/MyDrive/TTIC-Speech/dataset/dev-other.tar.gz

Streaming output truncated to the last 5000 lines.
LibriSpeech/train-clean-100/60/121082/60-121082-0057.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0061.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0085.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0038.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0073.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0093.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0044.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0037.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0024.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0087.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0075.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0058.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0039.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0011.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0049.flac
LibriSpeech/train-clean-100/60/121082/60-121082-0007.flac
LibriSpeech/train-cle

## Modify pre-trained checkpoint content

In [ ]:
%cd {root_dir}/SpeechLM_finetuning/SpeechLM/fairseq

import torch

model_path = '/content/drive/MyDrive/TTIC-Speech/pre-trained/speechlmp_base_checkpoint_298_400000.pt'

ckpt = torch.load(model_path)
ckpt['cfg']['task']['text_cfg']['text_data'] = '/content/SpeechLM_finetuning/SpeechLM/dataset/LibriLM/phone_unit/bin-idx/'
ckpt["cfg"]["model"]["freeze_layers"] = -1
torch.save(ckpt, model_path)
del ckpt

/content/SpeechLM_finetuning/SpeechLM/fairseq


# Fine-tune ASR

### Prepare data manifest

In [ ]:
%cd {root_dir}/SpeechLM_finetuning/SpeechLM/

dest_dir = 'dataset/LibriSpeech/asr'
!cp dataset/LibriLM/phone_unit/bin-idx/dict.ltr.txt {dest_dir}

/content/SpeechLM_finetuning/SpeechLM
cp: 'dataset/LibriLM/phone_unit/bin-idx/dict.ltr.txt' and 'dataset/LibriSpeech/asr/dict.ltr.txt' are the same file


In [ ]:
%cd {root_dir}/SpeechLM_finetuning/SpeechLM/fairseq/

libri_dir = {root_dir}/LibriSpeech

!python examples/wav2vec/wav2vec_manifest.py {libri_dir}/train-clean-100/ --dest ../{dest_dir} --ext "flac" --valid-percent 0
!mv ../{dest_dir}/train.tsv ../{dest_dir}/train_clean_100.tsv
!python examples/wav2vec/libri_labels.py ../{dest_dir}/train_clean_100.tsv --output-dir ../{dest_dir} --output-name=train_clean_100

!python examples/wav2vec/wav2vec_manifest.py {libri_dir}/dev-other/ --dest ../{dest_dir} --ext "flac" --valid-percent 0
!mv ../{dest_dir}/train.tsv ../{dest_dir}/dev_other.tsv
!python examples/wav2vec/libri_labels.py ../{dest_dir}/dev_other.tsv --output-dir ../{dest_dir} --output-name=dev_other

/content/SpeechLM_finetuning/SpeechLM/fairseq


### Fine-tune ASR on LibriSpeech 100hr training data

In [ ]:
%cd /content/SpeechLM_finetuning/SpeechLM/

model_path = '/content/drive/MyDrive/TTIC-Speech/pre-trained/speechlmp_base_checkpoint_298_400000.pt'
data_dir = '/content/SpeechLM_finetuning/SpeechLM/dataset/LibriSpeech/asr'
exp_name = 'test'

!bash speechlm/scripts/tune_speechlm_asr/finetune_base_ctc.sh {model_path} {data_dir} {exp_name}

/content/SpeechLM_finetuning/SpeechLM
2023-05-30 00:57:57.600766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-05-30 00:58:04,030][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': '/content/SpeechLM_finetuning/SpeechLM/exp/finetune_asr/pre-trained/ctc30k_from_test_bz1.6m_lr1e-5', 'wandb_project': None, 'azureml_logging': False, 'seed': 1337, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/conte

In [ ]:
!cp -r {root_dir}/SpeechLM_finetuning/SpeechLM/exp/finetune_asr/pre-trained/ctc30k_from_test_bz1.6m_lr1e-5 /content/drive/MyDrive/TTIC-Speech/exp/